In [1]:
import os
import numpy as np
from scipy.interpolate import griddata
import xarray as xr
import matplotlib.pyplot as plt
from scipy.ndimage import binary_erosion, binary_dilation
from scipy.spatial import cKDTree

In [2]:
# Répertoires fichiers grilles
rep = '/home/thibault-delahaye/git_dev/notebook-grid-tools/DATASETS_CROCOTOOLS/Topo/to_merge_ter/'

# Chemins des rasters à fusionner
input1 = rep + "MNT_NC100m_TSUCAL_GEO_refNM_ZNEG_V1.0.grd" # HIGH RESOLUTION
#input2 = rep + "gebco_2024_n-14.502_s-25.4004_w158.5986_e171.6064.nc" #LOW RESOLUTION
input2= rep + "etopo_2022_NC.tiff"

# Ouvrir le fichier 1
ds1 = xr.open_dataset(input1)
ds2 = xr.open_dataset(input2)
if input2[-4:]=='tiff':
    ds2= ds2.sel(band=1, drop=True)
# Identifier la variable contenant les données de bathymétrie
# On suppose que la variable de bathymétrie est la seule avec des valeurs numériques
for var_name in ds1.data_vars:
    var_data = ds1[var_name]
    # Vérifier si la variable est 2D et contient des valeurs numériques
    if var_data.ndim == 2 and var_data.dtype.kind in {'f', 'i'}:
         z_ds1= var_name
         break
else:
    print("Aucune variable de bathymétrie trouvée dans ds1")

for var_name in ds2.data_vars:
    var_data = ds2[var_name]
    # Vérifier si la variable est 2D et contient des valeurs numériques
    if var_data.ndim == 2 and var_data.dtype.kind in {'f', 'i'}:
         z_ds2= var_name
         break
else:
    print("Aucune variable de bathymétrie trouvée dans ds2")

# Identifier les noms des coordonnées lat/lon des fichiers

for coord_name in ds1.coords:
    coord_data = ds1[coord_name]
    # On suppose que lon et lat sont généralement des coordonnées avec des valeurs géographiques
    if coord_name.lower() in {'lon', 'longitude'}:
        lon_coord_ds1  = coord_name
    elif coord_name.lower() in {'lat', 'latitude'}:
        lat_coord_ds1  = coord_name
    elif coord_data.dims == ('x',) or coord_data.dims == ('y',):
        if 'x' in coord_data.dims:
            lon_coord_ds1 = coord_name
        if 'y' in coord_data.dims:
            lat_coord_ds1 = coord_name

for coord_name in ds2.coords:
    coord_data = ds2[coord_name]
    # On suppose que lon et lat sont généralement des coordonnées avec des valeurs géographiques
    if coord_name.lower() in {'lon', 'longitude'}:
        lon_coord_ds2  = coord_name
    elif coord_name.lower() in {'lat', 'latitude'}:
        lat_coord_ds2  = coord_name
    elif coord_data.dims == ('x',) or coord_data.dims == ('y',):
        if 'x' in coord_data.dims:
            lon_coord_ds2 = coord_name
        if 'y' in coord_data.dims:
            lat_coord_ds2 = coord_name

In [3]:
#Interpoler la grille de basse résolution sur une grille de résolution = HIGH RES

#Déterminer la résolution de la grille 1
resolution_lon= np.diff(ds1[lon_coord_ds1]).mean()
resolution_lat= np.diff(ds1[lat_coord_ds1]).mean()

lon2= ds2[lon_coord_ds2].values
lat2= ds2[lat_coord_ds2].values
z2= ds2[z_ds2].values

# Récupérér grille ds2 initiale
lon_grid_2, lat_grid_2 = np.meshgrid(lon2, lat2)

# Aplatir les grilles 2D et les valeurs
lon_flat_2 = lon_grid_2.ravel()
lat_flat_2 = lat_grid_2.ravel()
z_flat_2 = z2.ravel()

# Créer une nouvelle grille ds2
new_lon_2 = np.arange(lon2.min(), lon2.max(), resolution_lon)
new_lat_2 = np.arange(lat2.min(), lat2.max(), resolution_lat)
new_lon_grid_2, new_lat_grid_2 = np.meshgrid(new_lon_2, new_lat_2)

#INTERPOLATION
z2_interp = griddata((lon_flat_2, lat_flat_2), z_flat_2, (new_lon_grid_2, new_lat_grid_2), method='nearest')

In [4]:
lon1= ds1[lon_coord_ds1].values
lat1= ds1[lat_coord_ds1].values
z1= ds1[z_ds1].values

# Récupérér grille ds2 initiale
lon_grid_1, lat_grid_1 = np.meshgrid(lon1, lat1)

# Aplatir les grilles 2D et les valeurs
lon_flat_1 = lon_grid_1.ravel()
lat_flat_1 = lat_grid_1.ravel()
z_flat_1 = z1.ravel()

# 1. Identifier la zone de chevauchement entre les grilles
lon_min_1, lon_max_1 = lon1.min(), lon1.max()  # Étendue de la grille 1 en longitude
lat_min_1, lat_max_1 = lat1.min(), lat1.max()  # Étendue de la grille 1 en latitude

# Créer un masque sur la grille rééchantillonnée de la grille 2 qui couvre seulement la zone de la grille 1
mask_overlap = (new_lon_grid_2 >= lon_min_1) & (new_lon_grid_2 <= lon_max_1) & \
               (new_lat_grid_2 >= lat_min_1) & (new_lat_grid_2 <= lat_max_1)


In [5]:
# 2. Interpolation des données de la grille 1 (haute résolution) uniquement sur la zone couverte par la grille 1
z1_interp_on_z2_overlap = griddata(
    (lon_flat_1, lat_flat_1),  # Points d'origine (grille 1)
    z_flat_1,                  # Valeurs d'origine (grille 1)
    (new_lon_grid_2[mask_overlap], new_lat_grid_2[mask_overlap]),  # Points cibles (zone de la grille 1 dans la grille 2)
    method='nearest'           # Méthode d'interpolation
)

# REMPLACEMENT
z2_save= z2_interp.copy()

# 3. Remplacer les données de z2_interp uniquement dans la zone de chevauchement avec celles de la grille 1
z2_interp[mask_overlap] = z1_interp_on_z2_overlap

### LISSAGE

### Lissage des nan

In [7]:
def nan_buffer_linear_pond(blended_data, low_res_interp, buffer_width):
    # Create a mask for NaN values in blended_data where low_res_interp has values
    mask_nan = np.isnan(blended_data) & ~np.isnan(low_res_interp)
    new_z = blended_data.copy()
    
    # Create a structuring element of size buffer_width x buffer_width
    structure = np.ones((buffer_width, buffer_width), dtype=bool)  # Structuring element for erosion and dilation
    
    # Apply binary erosion to the NaN mask
    mask_eroded = binary_erosion(mask_nan, structure=structure)
    
    # Apply binary dilation to the eroded mask
    mask_dilated = binary_dilation(mask_eroded, structure=structure)
    
    # Compute the transition mask as the difference between dilated and eroded masks
    mask_transition = mask_dilated & ~mask_eroded
    
    # Compute the interpolation mask for areas not covered by the dilated mask
    mask_interpolation = mask_nan & ~mask_dilated
    
    # Apply binary dilation to the transition mask to expand it
    transition_dilated = binary_dilation(mask_transition, structure=np.ones((3, 3)))
    # Compute transition borders as the difference between expanded and transition masks
    transition_borders = transition_dilated & ~mask_transition
    
    # Assign low resolution data to the areas covered by the eroded mask
    new_z[mask_eroded] = low_res_interp[mask_eroded]  # Low-res data in the eroded zone
    
    # Get coordinates of transition and transition border areas
    coords_mask_transition = np.array(np.where(mask_transition)).T
    coords_transition_borders_int = np.array(np.where(transition_borders & mask_eroded)).T
    coords_transition_borders_ext = np.array(np.where(transition_borders & ~mask_eroded)).T
    
    # Build KDTree for internal and external transition borders
    tree_transition_borders_int = cKDTree(coords_transition_borders_int)
    tree_transition_borders_ext = cKDTree(coords_transition_borders_ext)
    
    # Compute minimum distances from mask transition coordinates to internal and external borders
    distances_int, indices_int = tree_transition_borders_int.query(coords_mask_transition)
    distances_ext, indices_ext = tree_transition_borders_ext.query(coords_mask_transition)
    
    def get_values_from_indices(indices, coords_borders):
        # Extract values from z2_combined based on indices
        values = np.full(len(indices), np.nan)
        for i, idx in enumerate(indices):
            if idx < len(coords_borders):
                x, y = coords_borders[idx]
                values[i] = new_z[x, y]
        return values
    
    # Get values from internal and external border coordinates
    values_int = get_values_from_indices(indices_int, coords_transition_borders_int)
    values_ext = get_values_from_indices(indices_ext, coords_transition_borders_ext)
    
    # Compute weighted average for transition values
    tot_dist = distances_int + distances_ext
    weight_out = distances_int / tot_dist
    weight_in = distances_ext / tot_dist
    ponderated_transition = weight_in * values_int + weight_out * values_ext
    
    def apply_ponderated_values(new_z, mask_transition, ponderated_values, coords_mask_transition):
        # Apply weighted values to z2_combined based on transition mask
        for i, (coord, value) in enumerate(zip(coords_mask_transition, ponderated_values)):
            x, y = coord
            if mask_transition[x, y]:
                new_z[x, y] = value
        return new_z
    
    # Apply weighted values to the combined matrix
    new_z = apply_ponderated_values(new_z, mask_transition, ponderated_transition, coords_mask_transition)

    return new_z, mask_transition

In [8]:
[z_less_nan, mask_buff_nan_10]= nan_buffer_linear_pond(z2_interp, z2_save, buffer_width=10)

In [ ]:
np.isnan(z_less_nan).sum()

In [9]:
z_less_nan_2, mask_buff_nan_3= nan_buffer_linear_pond(z_less_nan, z2_save, buffer_width=5)

In [10]:
np.isnan(z_less_nan_2).sum()

37085

In [11]:
z_less_nan_3, mask_buff_nan_3= nan_buffer_linear_pond(z_less_nan_2, z2_save, buffer_width=3)

In [12]:
np.isnan(z_less_nan_3).sum()

19752

In [13]:
z_less_nan_bis, mask_buff_nan_3= nan_buffer_linear_pond(z_less_nan_3, z2_save, buffer_width=2)

In [14]:
np.isnan(z_less_nan_bis).sum()

8911

In [15]:
8911/96357906

9.247814081804558e-05

### INTERPOLATION DES DERNIERS NAN

In [10]:
import pyinterp.backends.xarray
# Module that handles the filling of undefined values.
import pyinterp.fill

In [4]:
import pyinterp

# Données d'exemple
lon_flat_2 = np.ravel(new_lon_grid_2)
lat_flat_2 = np.ravel(new_lat_grid_2)
#z_flat_2 = np.ravel(z_less_nan_bis)

# Masque des NaN
#mask_nan = np.isnan(z_flat_2)

In [6]:
# Créer des axes pour les coordonnées
lon_axis = pyinterp.Axis(np.unique(lon_flat_2))
lat_axis = pyinterp.Axis(np.unique(lat_flat_2))

# Créer une grille 2D vide
grid_data = np.full((len(lat_axis), len(lon_axis)), np.nan)

In [24]:
data = pyinterp.Grid2D(lat_axis,lon_axis,z2_interp)

In [29]:
# Créer un objet Grid2D avec les axes
data = pyinterp.Grid2D(lat_axis,lon_axis,z_less_nan_bis)


In [23]:
z = pyinterp.fill.loess(data, nx=3, ny=3)

AttributeError: module 'pyinterp' has no attribute 'fill'

0

### LISSAGE DE LA ZONE TAMPON INTER_GRILLES (RECTANGLE)

#### Création du masque

In [54]:
import numpy as np

# Dimensions de la grille
grid_shape = new_lon_grid_2.shape

# Obtenir les indices des positions où le masque de recouvrement est True
true_indices = np.array(np.where(mask_overlap))

# Obtenir les coordonnées des 4 extrémités du mask_overlap
top = true_indices[0].min()   # Coordonnée de la première ligne avec True (haut)
bottom = true_indices[0].max()  # Coordonnée de la dernière ligne avec True (bas)
left = true_indices[1].min()  # Coordonnée de la première colonne avec True (gauche)
right = true_indices[1].max()  # Coordonnée de la dernière colonne avec True (droite)

# Largeur du tampon
buffer_width = 30  # Exemple
min_buffer_width = 3  # Largeur minimum du tampon

# Initialiser un masque tampon vide
mask_buffer = np.zeros(grid_shape, dtype=bool)

# Fonction pour créer un tampon si la distance au bord est suffisante
def create_buffer_on_side(start_idx, end_idx, buffer_limit, buffer_width, max_limit, side):
    if buffer_limit <= min_buffer_width:  # Pas assez de place pour un tampon propre
        print(f"Avertissement : Pas de place pour un buffer sur le côté {side}. Aucun lissage.")
        return 0
    elif buffer_limit < buffer_width:  # Réduire la taille du tampon
        print(f"Avertissement : Le buffer sur le côté {side} a été réduit à {buffer_limit} mailles.")
        buffer_width = buffer_limit
    return buffer_width

# Fonction pour vérifier les données numériques en dehors du tampon
def has_valid_data_outside(start_row, end_row, start_col, end_col):
    return np.any(new_lon_grid_2[start_row:end_row, start_col:end_col])

# Créer le tampon uniquement sur les côtés où le mask_overlap n'est pas trop près des bords
# et s'assurer qu'il y a de la donnée valide pour interpoler.

# Si le haut du mask_overlap n'est pas sur le bord haut de la grille
buffer_top = create_buffer_on_side(top, bottom, top, buffer_width, grid_shape[0], "haut")
if buffer_top > 0 and has_valid_data_outside(top-buffer_top, top, left, right+1): 
    mask_buffer[top - buffer_top:top, left:right+1] = True

# Si le bas du mask_overlap n'est pas sur le bord bas de la grille
buffer_bottom = create_buffer_on_side(bottom, bottom+buffer_width, grid_shape[0]-bottom-1, buffer_width, grid_shape[0], "bas")
if buffer_bottom > 0 and has_valid_data_outside(bottom+1, bottom+1+buffer_bottom, left, right+1): 
    mask_buffer[bottom+1:bottom+1+buffer_bottom, left:right+1] = True

# Si la gauche du mask_overlap n'est pas sur le bord gauche de la grille
buffer_left = create_buffer_on_side(left, right, left, buffer_width, grid_shape[1], "gauche")
if buffer_left > 0 and has_valid_data_outside(top, bottom+1, left-buffer_left, left): 
    mask_buffer[top:bottom+1, left-buffer_left:left] = True

# Si la droite du mask_overlap n'est pas sur le bord droit de la grille
buffer_right = create_buffer_on_side(right, right+buffer_width, grid_shape[1]-right-1, buffer_width, grid_shape[1], "droite")
if buffer_right > 0 and has_valid_data_outside(top, bottom+1, right+1, right+1+buffer_right): 
    mask_buffer[top:bottom+1, right+1:right+1+buffer_right] = True

# Inclure les coins si nécessaire
def include_corners(mask_buffer, top, bottom, left, right, buffer_width):
    # Coin supérieur gauche
    if top - buffer_width >= 0 and left - buffer_width >= 0:
        mask_buffer[top - buffer_width:top, left - buffer_width:left] = True
    # Coin supérieur droit
    if top - buffer_width >= 0 and right + buffer_width < grid_shape[1]:
        mask_buffer[top - buffer_width:top, right + 1:right + 1 + buffer_width] = True
    # Coin inférieur gauche
    if bottom + buffer_width < grid_shape[0] and left - buffer_width >= 0:
        mask_buffer[bottom + 1:bottom + 1 + buffer_width, left - buffer_width:left] = True
    # Coin inférieur droit
    if bottom + buffer_width < grid_shape[0] and right + buffer_width < grid_shape[1]:
        mask_buffer[bottom + 1:bottom + 1 + buffer_width, right + 1:right + 1 + buffer_width] = True

include_corners(mask_buffer, top, bottom, left, right, buffer_width)

# Exclure la zone de recouvrement du tampon
mask_buffer[mask_overlap] = False

# Afficher les coordonnées ajustées pour la zone tampon
print(f'Top: {top}, Bottom: {bottom}, Left: {left}, Right: {right}')


Avertissement : Pas de place pour un buffer sur le côté haut. Aucun lissage.
Top: 0, Bottom: 7379, Left: 2146, Right: 9045


#### Création des bordures du masque

In [55]:
dilated_mask_buffer= binary_dilation(mask_buffer)
border_buffer_ext = dilated_mask_buffer & ~mask_buffer & ~mask_overlap
border_buffer_inner = mask_overlap & dilated_mask_buffer

In [56]:
#LEBOn
# Get coordinates of borders and buffer
coords_border_inner = np.array(np.where(border_buffer_inner)).T
coords_border_outer = np.array(np.where(border_buffer_ext)).T
coords_buffer = np.array(np.where(mask_buffer)).T

# Get values from z2_interp at the border points
values_inner = z_less_nan_bis[border_buffer_inner]
values_outer = z_less_nan_bis[border_buffer_ext]

# Build KDTree for inner and outer borders
tree_border_inner = cKDTree(coords_border_inner)
tree_border_outer = cKDTree(coords_border_outer)

# Initialize array for interpolated values
interpolated_values_buffer = np.zeros(len(coords_buffer))

# For each point in the buffer, find nearest border points
for i, coord in enumerate(coords_buffer):
    # Query the KDTree for nearest points
    dist_inner, idx_inner = tree_border_inner.query(coord)
    dist_outer, idx_outer = tree_border_outer.query(coord)
    
    # Extract values of nearest border points
    value_inner = values_inner[idx_inner]
    value_outer = values_outer[idx_outer]
    
    # Calculate total distance and weights
    total_distance = dist_inner + dist_outer
    if total_distance == 0:
        # Handle the case where total distance is zero
        print(f"Warning: Total distance is zero for point {coord}")
        interpolated_values_buffer[i] = np.nan
    else:
        weight_inner = dist_outer / total_distance
        weight_outer = dist_inner / total_distance
        
        # Calculate interpolated value
        interpolated_values_buffer[i] = weight_inner * value_inner + weight_outer * value_outer

# Check for NaNs in interpolated_values_buffer
if np.any(np.isnan(interpolated_values_buffer)):
    print(f"NaNs found in interpolated_values_buffer")

In [57]:
z[mask_buffer] = interpolated_values_buffer

In [58]:
import xarray as xr

# Obtenez les coordonnées uniques
new_lon_unique = np.unique(new_lon_grid_2)
new_lat_unique = np.unique(new_lat_grid_2)

# Créer un DataArray pour les données interpolées
ds_interpolated = xr.DataArray(
    z, 
    coords=[('lat', new_lat_unique), ('lon', new_lon_unique)],  # Ajouter les coordonnées
    dims=['lat', 'lon']  # Spécifier les dimensions
)

In [59]:
# Créer un Dataset pour organiser les variables
ds_to_save = xr.Dataset({
    'z': ds_interpolated  # Ajouter la variable interpolée
})

In [61]:
ds_to_save.to_netcdf('composite_etopo_jroger_FINAL_3.nc', encoding={'z': {'chunksizes': (100, 100)}})


In [ ]:


# Créer un Dataset pour organiser les variables
ds_to_save = xr.Dataset({
    'z2_interp': ds_interpolated  # Ajouter la variable interpolée
})

# Sauvegarder dans un fichier NetCDF
ds_to_save.to_netcdf('composite_etopo_jroger_FINAL.nc')
